In [6]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [7]:
# get data
data = pd.read_csv('Level_1_NonPath_Path_GN_2286.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0     14.767060  10.829177  7.383530  ...      0.025877      0.025830      0.026292
1     11.728905   8.138424  3.351116  ...      0.025056      0.025025      0.026281
2      8.319776   2.773259  1.663955  ...      0.022828      0.023152      0.024468
3     10.255686   3.778411  2.428978  ...      0.023815      0.022707      0.024686
4      6.745884   2.698354  0.809506  ...      0.024331      0.024572      0.023173
...         ...        ...       ...  ...           ...           ...           ...
2281  12.828743   5.186088  4.640184  ...      0.024004      0.023269      0.024762
2282   6.040447   3.294790  1.372829  ...      0.025141      0.023511      0.020941
2283   3.997220   1.598888  2.931295  ...      0.026203      0.024164      0.024174
2284   3.739325   1.602568  2.938041  ...      0.026303      0.024191      0.024260
2285   9.393071   2.415361  4.830722  ...      0.023143      0.024699      0

In [8]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [9]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 2 - 1) = 1 components.
  ChangedBehaviorWarni

In [10]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 151,586
Trainable params: 151,586
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
58/58 [==============================] - 34s 58ms/step - loss: 0.6912 - accuracy: 0.5360 - val_loss: 0.6619 - val_accuracy: 0.6288
Epoch 2/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6664 - accuracy: 0.5879 - val_loss: 0.6523 - val_accuracy: 0.6179
Epoch 3/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6673 - accuracy: 0.5819 - val_loss: 0.6486 - val_accuracy: 0.6135
Epoch 4/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6736 - accuracy: 0.5685 - val_loss: 0.6505 - val_accuracy: 0.6157
Epoch 5/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6647 - accuracy: 0.5843 - val_loss: 0.6484 - val_accuracy: 0.6092
Epoch 6/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6702 - accuracy: 0.5728 - val_loss: 0.6497 - val_accuracy: 0.6157
Epoch 7/1000
58/58 [==============================] - 1s 10ms/step - loss: 0.6686 - accuracy: 0.5858 - val_loss: 0.6479 - val_accuracy: 0.615

In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.6028446389496718
test-acc = 0.6157205240174672
train-CFR =               precision    recall  f1-score   support

           0       0.63      0.57      0.60       945
           1       0.58      0.64      0.61       883

    accuracy                           0.60      1828
   macro avg       0.60      0.60      0.60      1828
weighted avg       0.61      0.60      0.60      1828

test-CFR =               precision    recall  f1-score   support

           0       0.66      0.56      0.61       243
           1       0.58      0.68      0.62       215

    accuracy                           0.62       458
   macro avg       0.62      0.62      0.62       458
weighted avg       0.62      0.62      0.62       458

